In [1]:
import os
import yaml
os.environ["BASE_DIR"] = "/work/FAC/FGSE/IDYST/tbeucler/downscaling"
BASE_DIR = os.environ["BASE_DIR"]
import json
import cartopy.crs as ccrs
import cartopy.feature as cfeature

/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvNew/lib/python3.10/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [2]:
import torch
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../../Scripts/Functions/Climate_Indices")

Loading the saved quick check trained model from Downscaling_Models/UNet_Deterministic_training_Dataset

In [3]:
model_path = os.path.join("full_best_model_huber_pretraining_FULL_RLOP.pth")
training_checkpoint =torch.load(model_path,map_location=torch.device('cpu')) #Moving model to CPU 

In [4]:
#Checking the parameters and keys
print(type(training_checkpoint))

<class 'dict'>


In [5]:
#Checking all model parameters 
for key in training_checkpoint.keys():
    print(key)

epoch
model_state_dict
optimizer_state_dict
loss


In [6]:
#Importing UNet class from Unet.py
sys.path.append(os.path.join(BASE_DIR, "sasthana/Downscaling/Downscaling_Models/models_UNet/UNet_Deterministic_Pretraining_Dataset"))
from UNet import UNet #Importing Unet class
from Downscaling_Dataset_Prep import DownscalingDataset #for creating paired frames

In [7]:
#Creating model instance

In [8]:
model_instance= UNet(in_channels=5, out_channels=4)
model_instance.load_state_dict(training_checkpoint["model_state_dict"])
model_instance.eval()

UNet(
  (Encoder1): Encoder_Block(
    (conv): DoubleConv(
      (conv): Sequential(
        (0): Conv2d(5, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (Encoder2): Encoder_Block(
    (conv): DoubleConv(
      (conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), b

In [9]:
#Scaled datasets for test set (2011-2020) :loading

In [10]:
# Scaling params loading from the .json files
scaling_dir = os.path.join(BASE_DIR, "sasthana/Downscaling/Downscaling_Models/Pretraining_Chronological_Dataset")
rhiresd_params = json.load(open(os.path.join(scaling_dir, "precip_scaling_params_chronological.json")))
tabsd_params   = json.load(open(os.path.join(scaling_dir, "temp_scaling_params_chronological.json")))
tmind_params   = json.load(open(os.path.join(scaling_dir, "tmin_scaling_params_chronological.json")))
tmaxd_params   = json.load(open(os.path.join(scaling_dir, "tmax_scaling_params_chronological.json")))

In [12]:
# Normalization functions : now using parameters from the loaded JSON files

def norm_precip(x, params):
    return (x - params["min"]) / (params["max"] - params["min"])

def norm_temp(x, params):
    return (x - params["mean"]) / params["std"]

def norm_tmin(x, params):
    return (x - params["mean"]) / params["std"]

def norm_tmax(x, params):
    return (x - params["mean"]) / params["std"]

In [13]:
precip=xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Processing_and_Analysis_Scripts/data_1971_2023/HR_files_full/RhiresD_1971_2023.nc")
temp=xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Processing_and_Analysis_Scripts/data_1971_2023/HR_files_full/TabsD_1971_2023.nc")
tmin=xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Processing_and_Analysis_Scripts/data_1971_2023/HR_files_full/TminD_1971_2023.nc")
tmax=xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Processing_and_Analysis_Scripts/data_1971_2023/HR_files_full/TmaxD_1971_2023.nc")

Renaming has to be done because the model was trained on the long time series and is being twstd on the observations of the shorter time series 

Inputs Prep

In [14]:
precip_input = xr.open_dataset(
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/RhiresD_step3_interp.nc"
).sel(time=slice("2011-01-01", "2020-12-31"))["RhiresD"].chunk({"time": 100}).rename("precip")
temp_input = xr.open_dataset(
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TabsD_step3_interp.nc"
).sel(time=slice("2011-01-01", "2020-12-31"))["TabsD"].chunk({"time": 100}).rename("temp")
tmin_input = xr.open_dataset(
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TminD_step3_interp.nc"
).sel(time=slice("2011-01-01", "2020-12-31"))["TminD"].chunk({"time": 100}).rename("tmin")
tmax_input = xr.open_dataset(
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/Training_Chronological_Dataset/TmaxD_step3_interp.nc"
).sel(time=slice("2011-01-01", "2020-12-31"))["TmaxD"].chunk({"time": 100}).rename("tmax")

In [15]:
# Normalize using loaded JSON parameters
precip_input = norm_precip(precip_input, rhiresd_params).rename("precip")
temp_input   = norm_temp(temp_input, tabsd_params).rename("temp")
tmin_input   = norm_tmin(tmin_input, tmind_params).rename("tmin")
tmax_input   = norm_tmax(tmax_input, tmaxd_params).rename("tmax")

Targets prep

In [16]:
precip_target = precip_target = norm_precip(
    precip.sel(time=slice("2011-01-01", "2020-12-31"))["RhiresD"].chunk({"time": 100}),
    rhiresd_params).rename("precip")
temp_target = norm_temp(
    temp.sel(time=slice("2011-01-01", "2020-12-31"))["TabsD"].chunk({"time": 100}),
    tabsd_params).rename("temp")
tmin_target = norm_tmin(
    tmin.sel(time=slice("2011-01-01", "2020-12-31"))["TminD"].chunk({"time": 100}),
    tmind_params).rename("tmin")
tmax_target = norm_tmax(
    tmax.sel(time=slice("2011-01-01", "2020-12-31"))["TmaxD"].chunk({"time": 100}),
    tmaxd_params).rename("tmax")

In [17]:
#Building the paired feature-target dataset; first loading individual and coverting them into images

In [18]:
# config used for training 
config_path = os.path.join(BASE_DIR, "sasthana/Downscaling/Downscaling_Models/models_UNet/UNet_Deterministic_Pretraining_Dataset/config.yaml")
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)


In [19]:
# elevation
elevation_path = os.path.join(BASE_DIR, "sasthana/Downscaling/Downscaling_Models/elevation.tif")


merging DS before creating pairs,,,,wont work for individual files

In [20]:
inputs_merged = xr.merge([precip_input, temp_input, tmin_input, tmax_input])
targets_merged = xr.merge([precip_target, temp_target, tmin_target, tmax_target])

ds = DownscalingDataset(inputs_merged, targets_merged, config, elevation_path)

Loaded elevation from /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/elevation.tif, shape: (255, 387)


In [21]:
#Checking shape of the ds instance 
print(f"Dataset shape: {ds.__len__()} samples") #Number of samples in the test set #Test set across configs remain the same 

Dataset shape: 3653 samples


In [22]:
#Checking shape of a random sample
input_img,target_img= ds[15] #14 th sample
print(f"Input image shape: {input_img.shape}")
print(f"Target image shape: {target_img.shape}")

Input image shape: torch.Size([5, 240, 370])
Target image shape: torch.Size([4, 240, 370])


In [23]:
from torch.utils.data import DataLoader

paired_ds = DataLoader(ds, batch_size=1, shuffle=False, num_workers=4)

In [24]:
idx= 2008
input_img, target_img =ds[idx]
input_img= input_img.unsqueeze(0) # Adding batch dimension
date=str(inputs_merged.time.values[idx]) #What date?


In [25]:
model_instance.eval()
with torch.no_grad():
    pred_img = model_instance(input_img).squeeze(0).cpu().numpy()  
target_img = target_img.cpu().numpy()  

In [27]:
def descale_precip(x, min_val, max_val):
    return x * (max_val - min_val) + min_val

def descale_temp(x, mean, std):
    return x * std + mean

In [28]:
pred_img_denorm = np.empty_like(pred_img)
target_img_denorm = np.empty_like(target_img)

pred_img_denorm[0] = descale_precip(pred_img[0], rhiresd_params["min"], rhiresd_params["max"])
pred_img_denorm[1] = descale_temp(pred_img[1], tabsd_params["mean"], tabsd_params["std"])
pred_img_denorm[2] = descale_temp(pred_img[2], tmind_params["mean"], tmind_params["std"])
pred_img_denorm[3] = descale_temp(pred_img[3], tmaxd_params["mean"], tmaxd_params["std"])

target_img_denorm[0] = descale_precip(target_img[0], rhiresd_params["min"], rhiresd_params["max"])
target_img_denorm[1] = descale_temp(target_img[1], tabsd_params["mean"], tabsd_params["std"])
target_img_denorm[2] = descale_temp(target_img[2], tmind_params["mean"], tmind_params["std"])
target_img_denorm[3] = descale_temp(target_img[3], tmaxd_params["mean"], tmaxd_params["std"])

In [ ]:
lat = inputs_merged.lat.values
lon = inputs_merged.lon.values
var_names = ["precip", "temp", "tmin", "tmax"]

# Define fixed colorbar limits for each variable
vmin_dict = {"precip": 0, "temp": -5, "tmin": -10, "tmax": -10}
vmax_dict = {"precip": 50, "temp": 25, "tmin": 20, "tmax": 30}

swiss_extent = [5.9, 10.5, 45.7, 47.9]  

fig, axes = plt.subplots(4, 3, figsize=(18, 18), subplot_kw={'projection': ccrs.PlateCarree()})
input_img = input_img.squeeze(0)  
for i, var in enumerate(var_names):
    # Get input image for this variable (input_img shape: (5, H, W))
    input_img_denorm = None
    if var == "precip":
        input_img_denorm = descale_precip(input_img[0].cpu().numpy(), rhiresd_params["min"], rhiresd_params["max"])
    elif var == "temp":
        input_img_denorm = descale_temp(input_img[1].cpu().numpy(), tabsd_params["mean"], tabsd_params["std"])
    elif var == "tmin":
        input_img_denorm = descale_temp(input_img[2].cpu().numpy(), tmind_params["mean"], tmind_params["std"])
    elif var == "tmax":
        input_img_denorm = descale_temp(input_img[3].cpu().numpy(), tmaxd_params["mean"], tmaxd_params["std"])

    # Use fixed vmin/vmax for each variable
    vmin = vmin_dict[var]
    vmax = vmax_dict[var]

    # Input
    ax = axes[i, 0]
    im = ax.pcolormesh(lon, lat, input_img_denorm, cmap='coolwarm', vmin=vmin, vmax=vmax, shading='auto', transform=ccrs.PlateCarree())
    ax.set_title(f"Input {var}")
    ax.set_extent(swiss_extent, crs=ccrs.PlateCarree())
    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.BORDERS)
    fig.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)

    # Target
    ax = axes[i, 1]
    im = ax.pcolormesh(lon, lat, target_img_denorm[i], cmap='coolwarm', vmin=vmin, vmax=vmax, shading='auto', transform=ccrs.PlateCarree())
    ax.set_title(f"Target {var}")
    ax.set_extent(swiss_extent, crs=ccrs.PlateCarree())
    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.BORDERS)
    fig.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)

    # Predicted
    ax = axes[i, 2]
    im = ax.pcolormesh(lon, lat, pred_img_denorm[i], cmap='coolwarm', vmin=vmin, vmax=vmax, shading='auto', transform=ccrs.PlateCarree())
    ax.set_title(f"Predicted {var}")
    ax.set_extent(swiss_extent, crs=ccrs.PlateCarree())
    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.BORDERS)
    fig.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
plt.suptitle(f"Inputs(bicubic) , Target(HR) and Predictions from the 1770s time series : {date}", fontsize=14, y=0.95)
plt.tight_layout()
plt.savefig(BASE_DIR+"/sasthana/Downscaling/Processing_and_Analysis_Scripts/Outputs/1_July_2016_unet_1771_2020.png", dpi=1000, bbox_inches='tight')
plt.show()